In [1]:
import codecs
import glob
import networkx as nx
import numpy as np
import os
import pandas as pd
import pickle
import re
from difflib import SequenceMatcher
from numpy import dot
from numpy.linalg import norm
from os import listdir
from os.path import isfile, join
from networkx.algorithms import isomorphism
from networkx.drawing.nx_pydot import write_dot
from utils import printProgressBar

In [ ]:
cow_dict = dict()
# counter = 0
with codecs.open('frequencies-frcowvec-filtered.csv', 'r', encoding='utf-8') as f:
    for line in f:
        elements = line.strip('\n').split(',')
        if elements[1] == 'freq':
            continue
        lexeme = elements[0].split('_')[0]
        try:
            if cow_dict[lexeme] < int(elements[1]):
                cow_dict[lexeme] = int(elements[1])
        except KeyError:
            cow_dict[lexeme] = int(elements[1])
#         print(counter, end='\r')
#         counter += 1
# print(counter)
print(len(cow_dict))

In [3]:
cow_set = set()
with codecs.open('frequencies-frcowvec-filtered.csv', 'r', encoding='utf-8') as f:
    for line in f:
        elements = line.strip('\n').split(',')
        if elements[1] == 'freq':
            continue
        cow_set.add(elements[0].split('_')[0])
print(len(cow_set))

11273094


In [4]:
demonette_lexemes = set()
with codecs.open('lexemes.csv', 'r', encoding='utf-8') as f:
    for line in f:
        elements = line.split('\t')
        if elements[0] == 'lid':
            continue
        demonette_lexemes.add(elements[2])
        
glawi_constructions = list()
with codecs.open('glawi-constructions.txt', 'r', encoding='utf-8') as f:
    for line in f:
        glawi_constructions.append(line.strip('\n'))
        
lexemes_in_bow = set()
with codecs.open('lemma-A-pos-bow.txt', 'r', encoding='utf-8') as f:
    for line in f:
        lexeme = line.split()[0].split('_')[0]
        lexemes_in_bow.add(lexeme)

In [5]:
def cosine_average(vector_dict, lex2):
    if len(vector_dict) == 0:
        return 0
    total_cosine = 0
    with codecs.open('lemma-A-pos-bow.txt', 'r', encoding='utf-8') as f:
        for line in f:
            lexeme = line.split()[0].split('_')[0]
            if lexeme == lex2:
                v2 = np.array(list(map(lambda x: float(x), line.split()[1:])))
                for v in vector_dict.values():
                    total_cosine += dot(v, v2)/(norm(v)*norm(v2))
                break
    return round(total_cosine / len(vector_dict), 2)

In [2]:
def match(pattern, word):  # pattern = 'preXisation', word = 'precognisation' => True
    if pattern == 'X':
        return True
    counter = 0
    try:
        for c in pattern:
            if c == 'X':
                break
            if c != word[counter]:
                return False
            counter += 1
        counter = -1
        while True:
            if pattern[counter] == 'X':
                break
            if pattern[counter] != word[counter]:
                return False
            counter -= 1
    except IndexError:  # Xtractif & actif
        return False
    return True

def generate_lexemes(nodes):
    return_dict = dict()
    return_list = list()  # list of tuples [(word, cosine)]
    lexemes = set()
    vector_dict = dict()
    count_not_in_bow = 0
    for n in nodes:
        lexeme = n.split('_')[0]
        if lexeme not in lexemes_in_bow and lexeme not in lexemes:
            count_not_in_bow += 1
        lexemes.add(lexeme)
#         vector_dict[lexeme] = np.array([])
    count_lexemes = len(lexemes) - count_not_in_bow
    with codecs.open('lemma-A-pos-bow.txt', 'r', encoding='utf-8') as f:
        for line in f:
            if count_lexemes <= 0:
                break
            lexeme = line.split()[0].split('_')[0]
            if lexeme in lexemes:
                vector_dict[lexeme] = np.array(list(map(lambda x: float(x), line.split()[1:])))
                count_lexemes -= 1
    for lexeme in lexemes:
        best_const_length = -1
        best_const = ''
        for c in glawi_constructions:
            const1 = c.split('-')[0]
            if len(const1) > best_const_length and match(const1, lexeme):
                best_const_length = len(const1)
                best_const = const1
        for c in glawi_constructions:
            [const1, const2] = c.split('-')
            if const1 != best_const:
                continue
            [prefix, postfix] = const1.split('X')
            stem = lexeme.replace(prefix, '', 1)
            if postfix:  # if not empty
                stem = ''.join(stem.rsplit(postfix, 1))
            new_lexeme = const2.replace('X', stem)
            if new_lexeme in return_dict or new_lexeme in lexemes:
                continue
            if new_lexeme in demonette_lexemes or new_lexeme in cow_set:
                if new_lexeme in lexemes_in_bow:
                    return_dict[new_lexeme] = cosine_average(vector_dict, new_lexeme)
                else:
                    return_dict[new_lexeme] = -2
                return_list.append((new_lexeme, return_dict[new_lexeme]))
#             try:
#                 return_dict[new_lexeme] = cow_dict[new_lexeme]
#             except KeyError:
#                 if new_lexeme in demonette_lexemes:
#                     return_dict[new_lexeme] = 0
    return_list.sort(key=lambda x:x[1], reverse=True)
    return return_list

In [6]:
binary_dir = 'demonette-glawinette_graph_binary'
input_files = [f for f in listdir(binary_dir) if isfile(join(binary_dir, f))]
input_files.sort()

In [ ]:
output_file = codecs.open('DG_propositions_and_cosineX.txt', 'a+', encoding='utf-8')
counter_file = -1
for input_file in input_files:
    print(input_file, end='\r')
    counter_file += 1
    if counter_file < 82:  #50
        continue
    G = pickle.load(open(join(binary_dir, input_file), 'rb'))
    output_file.write(input_file + '\t' + str(generate_lexemes(G.nodes())) + '\n')
output_file.close()

In [ ]:
output_file.close()

In [ ]:
G = pickle.load(open(join(binary_dir, 'F00082'), 'rb'))
generate_lexemes(G.nodes())

# reading vector space

In [ ]:
lexemes_in_bow = set()
with codecs.open('lemma-A-pos-bow.txt', 'r', encoding='utf-8') as f:
    for line in f:
        lexeme = line.split()[0].split('_')[0]
        lexemes_in_bow.add(lexeme)

In [ ]:
def cosine(lex1, lex2):
    if lex1 not in lexemes_in_bow or lex2 not in lexemes_in_bow:
        return -2
    found1 = False
    found2 = False
    with codecs.open('lemma-A-pos-bow.txt', 'r', encoding='utf-8') as f:
        for line in f:
            lexeme = line.split()[0].split('_')[0]
            if lexeme == lex1:
                print(line)
                v1 = np.array(list(map(lambda x: float(x), line.split()[1:])))
                found1 = True
                if found2:
                    break
            elif lexeme == lex2:
                print(line)
                v2 = np.array(list(map(lambda x: float(x), line.split()[1:])))
                found2 = True
                if found1:
                    break
    if found1 and found2:
        return dot(v1, v2)/(norm(v1)*norm(v2))
    else:
        return -2

In [ ]:
cosine('aimable', 'aimant')

# reading vector space to memory

In [ ]:
counter = -1
bow_dict = dict()
with codecs.open('lemma-A-pos-bow.txt', 'r', encoding='utf-8') as f:
    for line in f:
        counter += 1
        if counter == 0:
            print(line)
            continue
        if counter % 100 == 0:
            print(counter, end='\r')
        lexeme = line.split()[0].split('_')[0]
        v = np.array(list(map(lambda x: float(x), line.split()[1:])))
        bow_dict[lexeme] = v

In [ ]:
print('o')